In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## 1. Test scrapping page 1

In [20]:
base_url = 'https://momotor.id/motor-bekas?brands=suzuki'
r = requests.get(base_url)
r

<Response [200]>

In [21]:
soup = BeautifulSoup(r.content, 'html.parser')
content = soup.find_all('div', attrs={'class':'umcy-card-container'})
content

[<div class="umcy-card-container"><div class="component-umcy-card-catalog shadow-elevation-2 umcy-card-catalog" id="button_UmcyProductCardundefined"><div class="outer-img-wrapper"><div class="product-img-wrapper-catalog"><div class="component-image-slider-umcy"><div class="swiper-container"><div class="swiper-button-prev"></div><div class="swiper-button-next"></div><div class="swiper-pagination"></div><div class="swiper-wrapper"><div class="swiper-slide swiper-slide-duplicate" data-swiper-slide-index="4"><div class="umcy-item-image-container"><span style="box-sizing:border-box;display:block;overflow:hidden;width:initial;height:initial;background:none;opacity:1;border:0;margin:0;padding:0;position:absolute;top:0;left:0;bottom:0;right:0"><noscript><img alt="gambar" class="umcy-item-image" data-nimg="fill" decoding="async" id="img_UmcyProductCard4" loading="lazy" sizes="100vw" src="/_next/image?url=http%3A%2F%2Fres.cloudinary.com%2Fuploadz%2Fimage%2Fupload%2Fv1727709845%2Fmomotor%2Fprod%2

In [22]:
content[5].find('p', attrs={'id':'label_umcyProductundefined'})

<p class="subtitle-2" id="label_umcyProductundefined">GSX R 150</p>

## 2. Suzuki

In [23]:
listings = []
for i in content:
    title = i.find('p', attrs={'id':'label_umcyProductundefined'})
    price = i.find('p', attrs={'id':'label_umcyProductPriceundefined'})
    year = i.find('p', attrs={'id':'label_umcyProductYearundefined'})
    mileage = i.find('p', attrs={'id':'label_umcyProductMileageundefined'})
    location = i.find('p', attrs={'id':'label_umcyProductLocationundefined'})

    content_dict = {'vehicle_name': title,
                    'price': price,
                    'year':year,
                    'mileage':mileage,
                    'location':location}
    listings.append(content_dict)

pd.DataFrame(listings)

,vehicle_name,price,year,mileage,location
0,[GSX R 150],"[Rp15,300,000]",[2017],"[30,000 km]",[Padang Bulan]
1,[SATRIA F150 PREDATOR],"[Rp14,700,000]",[2016],"[30,000 km]",[Sidokumpul]
2,[NEX II],"[Rp11,100,000]",[2023],"[7,500 km]",[Gaharu]
3,[SATRIA FU 150],"[Rp12,600,000]",[2016],"[30,000 km]",[Kampung Tengah]
4,[SATRIA FU 150],"[Rp10,000,000]",[2018],"[30,000 km]",[Kota Depok]
5,[GSX R 150],"[Rp14,800,000]",[2017],"[15,000 km]",[Kota Cimahi]
6,[SATRIA FU 150],"[Rp12,600,000]",[2016],"[30,000 km]",[Kampung Tengah]
7,[SATRIA FU 150],"[Rp12,600,000]",[2016],"[30,000 km]",[Kampung Tengah]
8,[SATRIA FU 150],"[Rp11,400,000]",[2015],"[30,000 km]",[Kota Tangerang]
9,[GSX R 150],"[Rp14,800,000]",[2017],"[30,000 km]",[Kota Tangerang Selatan]


- Looping dengan Offset --> for offset in range(0, 216, 12) digunakan untuk mengiterasi nilai offset mulai dari 0 hingga 216, dengan 12 steps untuk setiap iterasi. Karena. struktur website momotor.id cukup unik, di mana base url nya adalah sebagai berikut (page 1) : 'https://momotor.id/motor-bekas?brands=suzuki';

- sementara, url untuk page 2 adalah: https://momotor.id/motor-bekas?offset=12
- url untuk page 3: https://momotor.id/motor-bekas?offset=24

dan seterusnya menggunakan offset=kelipatan 12

In [34]:
listings = []
for offset in range(0, 216, 12):
    url = f'{base_url}&offset={offset}'
    r = requests.get(url)
    
    # Memastikan permintaan berhasil
    if r.status_code == 200:
        soup = BeautifulSoup(r.content, 'html.parser')
        content = soup.find_all('div', attrs={'class':'umcy-card-container'})

        for i in content:
            title = i.find('p', attrs={'id':'label_umcyProductundefined'})
            price = i.find('p', attrs={'id':'label_umcyProductPriceundefined'})
            year = i.find('p', attrs={'id':'label_umcyProductYearundefined'})
            mileage = i.find('p', attrs={'id':'label_umcyProductMileageundefined'})
            location = i.find('p', attrs={'id':'label_umcyProductLocationundefined'})

            # Mengambil teks dari elemen yang ditemukan
            content_dict = {
                'vehicle_name': title.text if title else None,
                'price': price.text if price else None,
                'year': year.text if year else None,
                'mileage': mileage.text if mileage else None,
                'location': location.text if location else None
            }
            listings.append(content_dict)
    else:
        print(f'Error fetching page with offset {offset}: {r.status_code}')

suzuki_df = pd.DataFrame(listings)
suzuki_df

,vehicle_name,price,year,mileage,location
0,GSX R 150,"Rp15,300,000",2017,"30,000 km",Padang Bulan
1,SATRIA F150 PREDATOR,"Rp14,700,000",2016,"30,000 km",Sidokumpul
2,NEX II,"Rp11,100,000",2023,"7,500 km",Gaharu
3,SATRIA FU 150,"Rp12,600,000",2016,"30,000 km",Kampung Tengah
4,SATRIA FU 150,"Rp10,000,000",2018,"30,000 km",Kota Depok
...,...,...,...,...,...
204,SATRIA FU 150,"Rp12,600,000",2016,"30,000 km",Kota Pekanbaru
205,GSX R 150,"Rp14,300,000",2017,"30,000 km",Kab. Deli Serdang
206,GIXXER SF 250,"Rp38,500,000",2022,"15,000 km",Kab. Langkat
207,SATRIA FU 150,"Rp11,600,000",2015,"30,000 km",Kab. Magelang


In [38]:
suzuki_df['tax'] = 'hidup'
suzuki_df['seller_type'] = 'dealer'

In [39]:
suzuki_df['vehicle_name'].unique()

array(['GSX R 150', 'SATRIA F150 PREDATOR', 'NEX II', 'SATRIA FU 150',
       'GSX S 150', 'SATRIA 150 R', 'SATRIA 150', 'SATRIA F 150 CC',
       'ADDRESS FI 110 R', 'SATRIA F150 FIGH1AKS', 'FU  150 SC',
       'SATRIA BLACK FIRE', 'ADDRESS FI 110', 'GIXXER SF 250',
       'SATRIA F.150'], dtype=object)

In [42]:
suzuki_df['manufacturer'] = 'suzuki'
suzuki_df['model'] = suzuki_df['vehicle_name']

In [52]:
suzuki_df = suzuki_df.drop('vehicle_name', axis=1)

## 2. Honda

In [54]:
base_url_honda = 'https://momotor.id/motor-bekas?brands=honda'
listings = []
for offset in range(0, 9996, 12):
    url = f'{base_url_honda}&offset={offset}'
    r = requests.get(url)
    
    # Memastikan permintaan berhasil
    if r.status_code == 200:
        soup = BeautifulSoup(r.content, 'html.parser')
        content = soup.find_all('div', attrs={'class':'umcy-card-container'})

        for i in content:
            title = i.find('p', attrs={'id':'label_umcyProductundefined'})
            price = i.find('p', attrs={'id':'label_umcyProductPriceundefined'})
            year = i.find('p', attrs={'id':'label_umcyProductYearundefined'})
            mileage = i.find('p', attrs={'id':'label_umcyProductMileageundefined'})
            location = i.find('p', attrs={'id':'label_umcyProductLocationundefined'})

            # Mengambil teks dari elemen yang ditemukan
            content_dict = {
                'vehicle_name': title.text if title else None,
                'price': price.text if price else None,
                'year': year.text if year else None,
                'mileage': mileage.text if mileage else None,
                'location': location.text if location else None
            }
            listings.append(content_dict)
    else:
        print(f'Error fetching page with offset {offset}: {r.status_code}')

honda_df = pd.DataFrame(listings)
honda_df

,vehicle_name,price,year,mileage,location
0,NEW SCOOPY SPORTY,"Rp17,500,000",2022,"30,000 km",Kab. Lebak
1,VARIO 125 ESP CBS,"Rp20,500,000",2022,"30,000 km",Kota Denpasar
2,VARIO 125 CBS ISS,"Rp19,500,000",2022,"30,000 km",Kab. Lebak
3,NEW SCOOPY PRESTIGE,"Rp17,500,000",2021,"30,000 km",Kab. Lebak
4,VARIO 125 CBS ISS,"Rp15,500,000",2019,"30,000 km",Kab. Lebak
...,...,...,...,...,...
9991,SCOOPY ESP,"Rp15,800,000",2017,"30,000 km",Kota Surabaya
9992,NEW SUPRA X 125 CW,"Rp10,600,000",2016,"30,000 km",Kota Surabaya
9993,VARIO 150 ESP EXCLSV,"Rp17,500,000",2017,"30,000 km",Kota Denpasar
9994,BEAT CW,"Rp13,800,000",2021,"30,000 km",Kota Depok


In [72]:
honda_df = pd.DataFrame(listings)

In [73]:
honda_df['manufacturer']= 'Honda'
honda_df['tax']='hidup'
honda_df['seller_type']= 'dealer'

In [77]:
honda_df['model'] = honda_df['vehicle_name']
honda_df.drop(columns=['vehicle_name'], axis=1)

,price,year,mileage,location,manufacturer,tax,seller_type,model
0,"Rp17,500,000",2022,"30,000 km",Kab. Lebak,Honda,hidup,dealer,NEW SCOOPY SPORTY
1,"Rp20,500,000",2022,"30,000 km",Kota Denpasar,Honda,hidup,dealer,VARIO 125 ESP CBS
2,"Rp19,500,000",2022,"30,000 km",Kab. Lebak,Honda,hidup,dealer,VARIO 125 CBS ISS
3,"Rp17,500,000",2021,"30,000 km",Kab. Lebak,Honda,hidup,dealer,NEW SCOOPY PRESTIGE
4,"Rp15,500,000",2019,"30,000 km",Kab. Lebak,Honda,hidup,dealer,VARIO 125 CBS ISS
...,...,...,...,...,...,...,...,...
9991,"Rp15,800,000",2017,"30,000 km",Kota Surabaya,Honda,hidup,dealer,SCOOPY ESP
9992,"Rp10,600,000",2016,"30,000 km",Kota Surabaya,Honda,hidup,dealer,NEW SUPRA X 125 CW
9993,"Rp17,500,000",2017,"30,000 km",Kota Denpasar,Honda,hidup,dealer,VARIO 150 ESP EXCLSV
9994,"Rp13,800,000",2021,"30,000 km",Kota Depok,Honda,hidup,dealer,BEAT CW


## 3. Kawasaki

In [84]:
base_url_kawasaki = 'https://momotor.id/motor-bekas?brands=kawasaki'
listings = []
for offset in range(0, 804, 12):
    url = f'{base_url_kawasaki}&offset={offset}'
    r = requests.get(url)
    
    # Memastikan permintaan berhasil
    if r.status_code == 200:
        soup = BeautifulSoup(r.content, 'html.parser')
        content = soup.find_all('div', attrs={'class':'umcy-card-container'})

        for i in content:
            title = i.find('p', attrs={'id':'label_umcyProductundefined'})
            price = i.find('p', attrs={'id':'label_umcyProductPriceundefined'})
            year = i.find('p', attrs={'id':'label_umcyProductYearundefined'})
            mileage = i.find('p', attrs={'id':'label_umcyProductMileageundefined'})
            location = i.find('p', attrs={'id':'label_umcyProductLocationundefined'})

            # Mengambil teks dari elemen yang ditemukan
            content_dict = {
                'model': title.text if title else None,
                'price': price.text if price else None,
                'year': year.text if year else None,
                'mileage': mileage.text if mileage else None,
                'location': location.text if location else None
            }
            listings.append(content_dict)
    else:
        print(f'Error fetching page with offset {offset}: {r.status_code}')

kawasaki_df = pd.DataFrame(listings)
kawasaki_df

,model,price,year,mileage,location
0,D-TRACKER 150 SE,"Rp24,000,000",2021,"15,000 km",Kab. Lebak
1,W175,"Rp21,600,000",2019,"7,500 km",Kab. Bandung
2,NINJA RR 250 MONO,"Rp22,500,000",2020,"15,000 km",Kota Jakarta Utara
3,D-TRACKER 150 SE,"Rp30,300,000",2022,"15,000 km",Kab. Serang
4,D-TRACKER 150 SE,"Rp30,300,000",2022,"15,000 km",Kab. Serang
...,...,...,...,...,...
799,KLX 150 BF SE,"Rp28,400,000",2017,"30,000 km",Kota Pekanbaru
800,D TRACKER 150,"Rp26,000,000",2016,"30,000 km",Kota Pekanbaru
801,KLX 150 BF,"Rp27,200,000",2017,"30,000 km",Kota Pekanbaru
802,NEW NINJA 250 R,"Rp48,700,000",2019,"15,000 km",Kab. Magelang


In [85]:
kawasaki_df['manufacturer']= 'kawasaki'
kawasaki_df['tax']= 'hidup'
kawasaki_df['seller_type']= 'dealer'

In [86]:
kawasaki_df

,model,price,year,mileage,location,manufacturer,tax,seller_type
0,D-TRACKER 150 SE,"Rp24,000,000",2021,"15,000 km",Kab. Lebak,kawasaki,hidup,dealer
1,W175,"Rp21,600,000",2019,"7,500 km",Kab. Bandung,kawasaki,hidup,dealer
2,NINJA RR 250 MONO,"Rp22,500,000",2020,"15,000 km",Kota Jakarta Utara,kawasaki,hidup,dealer
3,D-TRACKER 150 SE,"Rp30,300,000",2022,"15,000 km",Kab. Serang,kawasaki,hidup,dealer
4,D-TRACKER 150 SE,"Rp30,300,000",2022,"15,000 km",Kab. Serang,kawasaki,hidup,dealer
...,...,...,...,...,...,...,...,...
799,KLX 150 BF SE,"Rp28,400,000",2017,"30,000 km",Kota Pekanbaru,kawasaki,hidup,dealer
800,D TRACKER 150,"Rp26,000,000",2016,"30,000 km",Kota Pekanbaru,kawasaki,hidup,dealer
801,KLX 150 BF,"Rp27,200,000",2017,"30,000 km",Kota Pekanbaru,kawasaki,hidup,dealer
802,NEW NINJA 250 R,"Rp48,700,000",2019,"15,000 km",Kab. Magelang,kawasaki,hidup,dealer


## 4. Gabungkan dengan Data OLX yang saya scrapping sebelumnya

In [158]:
olx = pd.read_csv(r'C:\Users\tamar\Desktop\Folder Latihan\olx_satu')
olx = olx.drop(columns=['Unnamed: 0', 'listing_title'], axis = 1)
olx['location']=olx['location'].apply(lambda x: x.split(',')[1])
honda_df=honda_df.drop('vehicle_name', axis=1)

In [195]:
data = pd.concat([olx,suzuki_df,honda_df,kawasaki_df]).reset_index(drop=True)
data

,price,location,manufacturer,model,year,mileage,tax,seller_type
0,Rp 15.300.000,Jakarta Selatan,Honda,Beat,2024,5.000-10.000 km,panjang4.,Diler
1,Rp 12.700.000,Jakarta Selatan,Honda,Vario,2016,0-5.000 km,hidupmotor,Individu
2,Rp 21.350.000,Jakarta Selatan,Honda,Scoopy,2024,0-5.000 km,juni,Diler
3,Rp 55.000.000,Jakarta Selatan,Yamaha,Xmax,2021,10.000-15.000 km,sampai,Individu
4,Rp 21.500.000,Jakarta Selatan,Yamaha,Aerox,2021,25.000-30.000 km,hidup,Individu
...,...,...,...,...,...,...,...,...
11173,"Rp28,400,000",Kota Pekanbaru,kawasaki,KLX 150 BF SE,2017,"30,000 km",hidup,dealer
11174,"Rp26,000,000",Kota Pekanbaru,kawasaki,D TRACKER 150,2016,"30,000 km",hidup,dealer
11175,"Rp27,200,000",Kota Pekanbaru,kawasaki,KLX 150 BF,2017,"30,000 km",hidup,dealer
11176,"Rp48,700,000",Kab. Magelang,kawasaki,NEW NINJA 250 R,2019,"15,000 km",hidup,dealer


## 5. Data Processing

### Price

In [183]:
data['price'] = data['price'].apply(lambda x: x.replace('Rp','').replace('.','').replace(',',''))
data['price'] = pd.to_numeric(data['price'], errors='coerce')
data['price']

0        15300000
1        12700000
2        21350000
3        55000000
4        21500000
           ...   
11173    28400000
11174    26000000
11175    27200000
11176    48700000
11177    24500000
Name: price, Length: 11178, dtype: int64

### Mileage

In [184]:
data['mileage'] = data['mileage'].apply(lambda x: str(x).split('-')[-2].replace(' km', '').replace('.', '')
                                        if '-' in str(x) else str(x).replace(' km', '').replace('.', ''))

In [185]:
data['mileage'] = data['mileage'].apply(lambda x: str(x).replace(',',''))
data['mileage']

0         5000
1            0
2            0
3        10000
4        25000
         ...  
11173    30000
11174    30000
11175    30000
11176    15000
11177    15000
Name: mileage, Length: 11178, dtype: object

In [187]:
data['mileage'] = pd.to_numeric(data['mileage'], errors='coerce')
data['mileage']

0         5000.0
1            0.0
2            0.0
3        10000.0
4        25000.0
          ...   
11173    30000.0
11174    30000.0
11175    30000.0
11176    15000.0
11177    15000.0
Name: mileage, Length: 11178, dtype: float64

### Manufacturer

In [188]:
data['manufacturer'].value_counts()

manufacturer
Honda                      10059
kawasaki                     804
suzuki                       209
Yamaha                        31
Kawasaki                      25
Piaggio                       22
Lain-lain                      8
Harley Davidson                5
KTM                            4
Triumph                        2
Benelli                        2
Suzuki                         2
TVS                            1
Off-Road dan Auto-Cross        1
Kymco                          1
BMW                            1
Viar                           1
Name: count, dtype: int64

In [189]:
data['manufacturer'] = data['manufacturer'].str.lower()
data['manufacturer'].value_counts()

manufacturer
honda                      10059
kawasaki                     829
suzuki                       211
yamaha                        31
piaggio                       22
lain-lain                      8
harley davidson                5
ktm                            4
triumph                        2
benelli                        2
viar                           1
tvs                            1
off-road dan auto-cross        1
kymco                          1
bmw                            1
Name: count, dtype: int64

### Seller Type

In [190]:
data['seller_type'].value_counts()

seller_type
dealer      11009
Individu      115
Diler          38
unknown        16
Name: count, dtype: int64

In [191]:
data[data['seller_type']=='Diler'] = data[data['seller_type']=='dealer']

In [192]:
data['seller_type'].value_counts()

seller_type
dealer      11009
Individu      115
unknown        16
Name: count, dtype: int64

## Final Cleaned Data

In [199]:
data

,price,location,manufacturer,model,year,mileage,tax,seller_type
0,Rp 15.300.000,Jakarta Selatan,Honda,Beat,2024,5.000-10.000 km,panjang4.,Diler
1,Rp 12.700.000,Jakarta Selatan,Honda,Vario,2016,0-5.000 km,hidupmotor,Individu
2,Rp 21.350.000,Jakarta Selatan,Honda,Scoopy,2024,0-5.000 km,juni,Diler
3,Rp 55.000.000,Jakarta Selatan,Yamaha,Xmax,2021,10.000-15.000 km,sampai,Individu
4,Rp 21.500.000,Jakarta Selatan,Yamaha,Aerox,2021,25.000-30.000 km,hidup,Individu
...,...,...,...,...,...,...,...,...
11173,"Rp28,400,000",Kota Pekanbaru,kawasaki,KLX 150 BF SE,2017,"30,000 km",hidup,dealer
11174,"Rp26,000,000",Kota Pekanbaru,kawasaki,D TRACKER 150,2016,"30,000 km",hidup,dealer
11175,"Rp27,200,000",Kota Pekanbaru,kawasaki,KLX 150 BF,2017,"30,000 km",hidup,dealer
11176,"Rp48,700,000",Kab. Magelang,kawasaki,NEW NINJA 250 R,2019,"15,000 km",hidup,dealer
